## [NYCDSA Capstone Project] 
# Women's Softball League Power Ranking Estimate

<br>
Koeun Lim (koeunlim@alum.mit.edu)<br>
Kevin Haghi (kevin.haghi@gmail.com)<br>


# Step 8. Visualization & Dashboard

---
## Project Description



### Project Outline
- Step 1. Web scraping
- Step 2. Clean data
- Step 3. EDA
- Step 4. Imputation & PCA
- Step 5. Modeling (1) RPI prediction based on stats - Linear regression
- Step 6. Modeling (2) WS 64 Seed prediction based on stats - Logistic regression
- Step 7. Modeling (3) WS Ranking prediction based on stats - Multi-class Logistic regression
- Step 8. Visualization & Dashboard

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

from jupyter_dash import JupyterDash
import dash_bootstrap_components as dbc

import dash
import dash_table
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State

import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
import plotly.figure_factory as ff

import seaborn as sns

from matplotlib import pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (20, 5)
pd.set_option('display.max_columns', 250)
pd.set_option('display.max_rows', 250)

In [2]:
# When running in JupyterHub or Binder, call the infer_jupyter_config function to detect the proxy configuration.
#JupyterDash.infer_jupyter_proxy_config()

### Load data & Preprocess

In [3]:
### Load current stats
df_all = pd.read_csv('Data/Clean_final.csv')
df_2019 = df_all[df_all.Year == 2019]
df_2019['Hit_Batters'] = -df_2019['Hit_Batters']
df_2019['Earned_Run_Average'] = -df_2019['Earned_Run_Average']

#################################
# Reorder to display as desired #
#################################
to_reorder = df_2019.columns.tolist()
to_reorder = [to_reorder[10], to_reorder[21], to_reorder[16], to_reorder[18], \
 to_reorder[22], to_reorder[14], to_reorder[15], to_reorder[19], to_reorder[13], to_reorder[20], \
 to_reorder[23], to_reorder[17], to_reorder[12], to_reorder[9],to_reorder[11]]    
df_2019_ord = df_2019[to_reorder]
scaler = StandardScaler()
df_2019_norm = pd.concat([df_2019.College,\
                          pd.DataFrame(scaler.fit_transform(df_2019_ord))],axis=1)
df_2019_norm.columns = ['College'] + list(df_2019_ord.columns)

df_norm = pd.DataFrame(scaler.fit_transform(df_2019_ord))
df_norm = pd.DataFrame(df_norm.stack()).reset_index().drop(columns=['level_0'])
df_norm.columns = ['Stats','Score']

indicator_college = np.sort(df_2019.College.unique())

### Load RPI prediction
df_RPI = pd.read_csv('RPI/RPI_prediction_logit_1000.csv')

### Load WS 64 Seeds prediction
df_WS64 = pd.read_csv('WS64Seeds/WS64_prediction_SGDlogistic_1000_famd_ext2.csv')

### Load WS Ranking prediction
df_WSProb = pd.read_csv('WSRank/WSProb_prediction_logistic_1000_famd_stratified.csv')
df_WSProb = df_WSProb.set_index(np.tile(df_RPI.columns,1000))
df_WSRank = pd.read_csv('WSRank/WSRank_prediction_logistic_1000_famd_stratified.csv')
df_WSRank = df_WSRank.set_index(np.tile(df_RPI.columns,1000))

In [4]:
####################
# Historical Stats #
####################
df_all = df_all[df_all.Year < 2019]
RPI_rank = df_all['RPI_Ranking']
teams = df_all['College'].unique()
rank = df_all['WCWS_Rank']
team_columns = ['College','Ave_WCWS_Rank']
team_rank = np.zeros((len(teams)))
team_high = np.zeros((len(teams)))
team_first = np.zeros((len(teams)),dtype = int)
team_NCAA = np.zeros((len(teams)))
team_WCWS = np.zeros((len(teams)))
team_RPI_high = np.zeros((len(teams)))

for i,team in enumerate(teams):
    team_rank[i] = np.round(np.mean(rank[df_all['College']==team]),2) #Average finish
    team_high[i] = np.min(rank[df_all['College']==team]) #highest finish
    team_RPI_high[i] = int(np.min(RPI_rank[df_all['College']==team])) #highest RPI
    team_first[i] = int(len([j for j in rank[df_all['College']==team] if j == 1]))
    team_NCAA[i] = int(len([j for j in rank[df_all['College']==team] if j <= 64]))
    team_WCWS[i] = int(len([j for j in rank[df_all['College']==team] if j <= 8]))
    
Ranked_WCWS = [x for _, x in sorted(zip(team_rank, teams))]
Top_Ten = Ranked_WCWS[0:10]
#print(int(team_high[np.where(teams == 'Oklahoma')]))

In [5]:
stats_list = df_norm.Stats.unique()
N = len(df_norm.Stats.unique())

#c = ['#F3BD96','#F3B296','#F3A796','#F39C96','#F3969B','#F396A6','#F396B1','#F396BC',
#     '#78BCC2','#78C2BE','#78C2B5','#78C2AD','#78C2A4','#78C29B','#F2D14B']
c = ['#997a8d','#a978ad','#d183b5','#d89bbc','#f4cdcc','#e99b9a','#dc6c64',
     '#dda982','#2a4d69','#638ec1','#6c94cc','#63ace5','#55b3d0','#80c5d0','#f2d14b']
#c = ['hsl('+str(h)+',72%'+',72%)' for h in np.linspace(0, 360, N)]

In [6]:
def get_250(df_sorted,rpi_predict):
    idx_rpi = df_sorted.index[df_sorted['index'] == rpi_predict][0]
    df_sorted.iloc[idx_rpi,1] = df_sorted.iloc[idx_rpi,1]/2
    
    n_sum_upper = 0
    i_upper = 1
    while n_sum_upper < 250:
        n_sum_upper = df_sorted.iloc[idx_rpi:idx_rpi+i_upper,1].sum()        
        i_upper += 1
        
        if (idx_rpi+i_upper) > len(df_sorted):
            n_sum_upper = 250
        #print(n_sum_upper)
    
    n_sum_lower = 0
    i_lower = 0
    while n_sum_lower < 250:
        n_sum_lower = df_sorted.iloc[idx_rpi-i_lower:idx_rpi+1,1].sum()        
        i_lower += 1
        
        if (idx_rpi-i_lower) <= 0:
            n_sum_lower = 250
        #print(n_sum_lower)
    
    #print(idx_rpi+i_upper)
    #print(idx_rpi-i_lower)
    return df_sorted['index'][idx_rpi+i_upper-2], df_sorted['index'][idx_rpi-i_lower+1]

### Construct the App

In [7]:
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.FLATLY])

######################################################################################################################
### Navbar components 
######################################################################################################################
SB_LOGO = "https://www.ncaa.com/modules/custom/casablanca_core/img/sportbanners/softball.png"

navbar = dbc.Navbar(
    [
        html.A(
            # Use row and col to control vertical alignment of logo
            dbc.Row(
                [
                    dbc.Col(html.Img(src=SB_LOGO, height="30px")),
                    dbc.Col(dbc.NavbarBrand("Women's College Softball Projections", className="ml-2")),
                ],
                align="center",
                no_gutters=True,
            ),
        ),
        dbc.NavbarToggler(id="navbar-toggler"),
        dbc.Collapse(
                dbc.Nav(
                    [
                        dbc.DropdownMenu(
                            [
                                dbc.DropdownMenuItem("Data", header=True),
                                dbc.DropdownMenuItem("Download"),
                                dbc.DropdownMenuItem(divider=True),
                                dbc.DropdownMenuItem("External Links", header=True),
                                dbc.DropdownMenuItem("NCAA",href="https://www.ncaa.com/sports/softball/d1"),
                                dbc.DropdownMenuItem("World Series",href="https://www.ncaa.com/news/softball/article/2020-05-04/womens-college-world-series-2021-schedule"),
                            ],
                            nav=True,
                            in_navbar=True,
                            label="More to Explore",
                        ),
                    ],
                    className="ml-auto", 
                    navbar=True,
                ),
                id="navbar-collapse",
                navbar=True,
            ),
    ],
    className = "navbar navbar-expand-lg navbar-dark",
    color="primary",
    #dark=True,
    #className="mb-1",
)

######################################################################################################################
### Sidebar components 
######################################################################################################################
SIDEBAR_STYLE = {
    "padding": "2rem 1rem 50rem",
    "background-color": '#e4f5f2',
}

hist_card = dbc.Card(id='Card_Update')

sidebar = html.Div(
    [
        html.H6("Select a college", id='college_out'),
        html.Hr(),
        dcc.Dropdown(
            id='college-dropdown',
            options=[{'label': i, 'value': i} for i in indicator_college],
            placeholder = 'Select a team'
        ),
        html.Br(),
        dbc.ListGroup([
            dbc.ListGroupItem(hist_card,color = '#e4f5f2'),
        ], flush=True,
        ),
    ],
    style=SIDEBAR_STYLE,
)

######################################################################################################################
### Main content components
######################################################################################################################
CONTENT_STYLE = {
    "padding": "1rem 1rem",
}

####################################
# Content 1: Current stats summary #
#----------------------------------#----------------------------------------------------------------------------------
fig1_default = go.Figure(data=[go.Box(
    y=df_2019_norm.iloc[:,i+1],marker_color=c[i],boxpoints=False,
    name=df_2019_norm.columns[i+1].replace('_',' ')) for i in range(int(N))])
fig1_default.update_layout(
    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    yaxis=dict(zeroline=True, gridcolor='white',
               tickmode='array',tickvals=[-3, -1.64, 0, 1.64, 3],
               ticktext=['Worse  ','(Bottom 5%)', 'Average', '(Top 5%)', 'Better  ']),
    paper_bgcolor="#f7f7f9",
    plot_bgcolor='#f7f7f9',
    height=400,
    yaxis_range=[-6,6],
    margin=dict(
        l=80,
        r=50,
        b=50,
        t=50,
        pad=4
    ),
)
fig1_default

tab1_content = dbc.Card([
    html.H5(dbc.CardHeader("Performance Stats Distribution: All Teams",
                           id='stats_description',className='card-header text-danger strong')),
    dbc.CardBody(
        [
            dcc.Graph(id='current_stats',figure=fig1_default)
        ],
        id='tab1_stats',
        style=CONTENT_STYLE,
    ),
    dbc.CardFooter("WARM colors show offensive stats and COOL colors show defensive stats.", 
                   className='text-muted small'),
],
className="mt-3 mr-4",
)

#############################
# Content 2: RPI Projection #
#---------------------------#-----------------------------------------------------------------------------------------
tab2_content = dbc.Card([
    html.H5(dbc.CardHeader("RPI Ranking Projection",
                           id='RPI_description',className='card-header text-danger strong')),
    dbc.CardBody(
        [
            dbc.Row([
                dbc.Col([
                    html.H4(id='RPI_rank',className="text-center text-info"),
                    html.P(id='RPI_range',className="text-center"),
                ],width=3,align='center'),
                dbc.Col(dcc.Graph(id='RPI_projection')),
            ]),
        ],
        id='tab2_RPI',
        style=CONTENT_STYLE,
    ), 
],
className="mt-3 mr-4",
)

############################
# Content 3: WS Projection #
#--------------------------#------------------------------------------------------------------------------------------
tab3_content = html.Div([
    dbc.Card([
        html.H5(dbc.CardHeader("NCAA Tournament of 64: Seeding chance",
                           id='NCWA_seeding_description',className='card-header text-danger strong')),
        dbc.CardBody([
            dbc.Row([
                dbc.Col([
                    html.H5(id='NCWA_seeding_result',className='text-center text-info'),
                    html.Br(),
                    html.P(id='NCWA_seeding_prob',className="text-center"),
                    html.P(id='NCWA_seeding_odds',className="text-center"),
                    html.P('*100 max, >1 indicates greater odds making.', className="text-center small em")
                ],width=6,align='center'),
                dbc.Col(dcc.Graph(id='NCWA_seeding_projection')),
            ]),
        ],
        id='tab3_NCWA_seeding',
        style=CONTENT_STYLE,
        ),
    ],
    className="mt-3 mr-4",
    ),
    dbc.Card([
    html.H5(dbc.CardHeader("NCAA & WS Performance",
                           id='NCWA_ranking_rescription',className='card-header text-danger strong')),
        dbc.CardBody([
            dbc.Row([
                dbc.Col([
                    dcc.Graph(id='NCWA_ranking_projection'),
                    html.Br(),
                    html.Div(id='NCWA_ranking_table'),
                ],width=8),
                dbc.Col([
                    html.H5(id='NCWA_ranking_result',className='card-title text-center text-info'),
                    html.Br(),
                    html.P(id='NCWA_ranking_prob',className="card-text text-center"),
                    html.Br(),
                    html.Hr(),
                    html.Br(),
                    html.H5(id='WS_seeding_result',className='card-title text-center text-info'),
                    html.Br(),
                    html.P(id='WS_seeding_prob',className="card-text text-center"),
                    html.P(id='WS_seeding_odds',className="card-text text-center"),
                    html.P('*100 max, >1 indicates greater odds making.', className="text-center small em"),
                ],align='center'),
            ]),
            
        ],
        id='tab3_NCWA_ranking',
        style=CONTENT_STYLE,
        ),
    ],
    className="mt-3 mr-4",
    )
])

###############
# Define Tabs #
#-------------#-------------------------------------------------------------------------------------------------------
tabs = dbc.Tabs(
    [
        dbc.Tab(tab1_content, label="Current Stats"),
        dbc.Tab(tab2_content, label="RPI Projection"),
        dbc.Tab(tab3_content, label="NCAA/WS Tournament Projection"),
    ],
    className='mt-3 mr-4',
)


######################################################################################################################
### Construct App
######################################################################################################################
app.layout = html.Div([
    dbc.Row(dbc.Col(navbar)),
    dbc.Row(
        [
            dbc.Col(sidebar,width=3),
            dbc.Col(tabs),
        ],
        no_gutters=False,
    )
])


# Navbar toggler
@app.callback(
    Output("navbar-collapse", "is_open"),
    [Input("navbar-toggler", "n_clicks")],
    [State("navbar-collapse", "is_open")],
)
def toggle_navbar_collapse(n, is_open):
    if n:
        return not is_open
    return is_open


# Stats Summary
@app.callback(
    Output('stats_description','children'),    # tab1         text1
    Output("current_stats", "figure"),         #              fig1
    Output('RPI_description','children'),      # tab2         text2
    Output('RPI_projection','figure'),         #              fig2
    Output('RPI_rank','children'),             #        card  text2_1
    Output('RPI_range','children'),            #              text2_2
    Output('NCWA_seeding_result','children'),  # tab3-1 card  text3_1_1
    Output('NCWA_seeding_prob','children'),    #              text3_1_2
    Output('NCWA_seeding_odds','children'),    #              text3_1_3
    Output('NCWA_seeding_projection','figure'),#              fig3_1
    Output('NCWA_ranking_result','children'),  # tab3-2 card1 text3_2_1
    Output('NCWA_ranking_prob','children'),    #              text3_2_2
    Output('WS_seeding_result','children'),    # tab3-2 card2 text3_2_3
    Output('WS_seeding_prob','children'),      #              text3_2_4
    Output('WS_seeding_odds','children'),      #              text3_2_5
    Output('NCWA_ranking_projection','figure'),#              fig3_2
    Output('NCWA_ranking_table','children'),   #              table3
    Output('Card_Update','children'),          # Sidebar History contents
    [Input('college-dropdown', 'value')],
)
def update_content(college):
    ################
    # No Selection #                                                                                      No Selection
    #==============#==================================================================================================
    if not college:
        ################
        # Tab1 content #                                                                           No Selection - Tab1
        #--------------#----------------------------------------------------------------------------------------------
        text1 = "Performance Stats Distribution: All Teams"
        fig1 = fig1_default
        
        ################
        # Tab2 content #                                                                           No Selection - Tab2
        #--------------#----------------------------------------------------------------------------------------------
        text2 = "RPI Ranking Projection"
        fig2 = fig1_default
        text2_1 = ' '
        text2_2 = ' '
        
        ################
        # Tab3 content #                                                                           No Selection - Tab3
        #--------------#----------------------------------------------------------------------------------------------
        text3_1_1 = " "
        text3_1_2 = " "
        text3_1_3 = " "
        fig3_1 = fig1_default
        text3_2_1 = " "
        text3_2_2 = " "
        text3_2_3 = " "
        text3_2_4 = " "
        text3_2_5 = " "
        fig3_2 = fig1_default
        table3 = [dbc.Table()]
        
        ###################
        # Sidebar content #                                                                     No Selection - history
        #-----------------#-------------------------------------------------------------------------------------------
        hist_new = dbc.Card(id='Card_Update')
        
        return (text1, fig1, text2, fig2, text2_1, text2_2, text3_1_1, text3_1_2, text3_1_3, fig3_1,
                text3_2_1, text3_2_2, text3_2_3, text3_2_4, text3_2_5, fig3_2, table3, hist_new)            #Return No Selection
    #=================================================================================================================
    
    
    ####################
    # College Selected #                                                                              College Selected
    #==================#==============================================================================================
    else:
        ################
        # Tab1 content #                                                                       College Selected - Tab1
        #--------------#----------------------------------------------------------------------------------------------
        text1 = "Average Performance Stats: {}".format(college)
        filtered_df = df_2019_norm[df_2019_norm.College == college].iloc[:,1:16]
        fig1 = go.Figure(
            data=[go.Bar(y=filtered_df.iloc[:,i],marker_color=c[i],name=filtered_df.columns[i].replace('_',' ')) for i in range(int(N))])
    
        fig1.update_layout(
            xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
            yaxis=dict(zeroline=True, gridcolor='white',
                       tickmode='array',tickvals=[-3, -1.64, 0, 1.64, 3],
                       ticktext=['Worse  ','(Bottom 5%)', 'Average', '(Top 5%)', 'Better  ']),
            paper_bgcolor="#f7f7f9",
            plot_bgcolor='#f7f7f9',#yaxis_title='Relative Score',
            height=400,
            yaxis_range=[-5,5],
            margin=dict(
                l=80,
                r=50,
                b=50,
                t=50,
                pad=4
            ),
        )
        
        ################
        # Tab2 content #                                                                       College Selected - Tab2
        #--------------#----------------------------------------------------------------------------------------------
        text2 = 'RPI Rank Projection: {}'.format(college)
        fig2 = ff.create_distplot([np.reshape(list(df_RPI[college]),(1,-1))[0]],group_labels=[college],
                                  show_rug=False,show_curve=False,bin_size=1,colors=['#ffa88f'])
        fig2.update_layout(
            xaxis=dict(showgrid=False, zeroline=False, showticklabels=True),
            yaxis=dict(zeroline=True, gridcolor='white'),
            xaxis_title='Ranking',
            yaxis_title='Likelihood',
            xaxis_range=(np.floor(df_RPI[college].min()/10)*10,np.ceil(df_RPI[college].max()/10)*10),
            paper_bgcolor="#f7f7f9",
            plot_bgcolor='#f7f7f9',
            height=300,
            width=500,
            #yaxis_range=[0,1],
            margin=dict(
                l=70,
                r=40,
                b=70,
                t=40,
                pad=4
            ),
        )
        
        RPI_predict = df_RPI[college].value_counts().idxmax()
        RPI_sorted = df_RPI[college].value_counts().sort_index(ascending=False).reset_index()
        RPI_bounds = get_250(RPI_sorted,RPI_predict)
        
        text2_1 = 'Rank {}'.format(int(df_RPI[college].value_counts().idxmax()))
        text2_2 = 'Range {}~{}'.format(int(RPI_bounds[0]),int(RPI_bounds[1]))

        ################
        # Tab3 content #                                                                       College Selected - Tab3
        #--------------#----------------------------------------------------------------------------------------------
        data_WS64 = np.reshape(list(df_WS64[college]),(1,-1))[0]*100
        fig3_1 = ff.create_distplot([data_WS64],group_labels=[college],
                                  show_rug=False,show_curve=False,bin_size=1,colors=['#ffa88f'])
        fig3_1.update_layout(
            xaxis=dict(showgrid=False, zeroline=False, showticklabels=True),
            yaxis=dict(zeroline=True, gridcolor='white'),
            xaxis_title='% Chance Making into NCAA Tournament',
            yaxis_title='Likelihood',
            xaxis_range=[0,100],#(np.floor(df_RPI[college].min()/10)*10,np.ceil(df_RPI[college].max()/10)*10),,
            #yaxis_range=[0,1],
            paper_bgcolor="#f7f7f9",
            plot_bgcolor='#f7f7f9',
            height=250,
            width=450,
            margin=dict(
                l=70,
                r=40,
                b=70,
                t=40,
                pad=4
            ),
        )
        mode_NCWA_prob = df_WS64[college].value_counts().idxmax()
        odds_NCWA = np.round(mode_NCWA_prob/(1-mode_NCWA_prob+0.000001),decimals=2)
        if odds_NCWA > 100:
            odds_NCWA = 100
            
        if odds_NCWA >= 1:
            text3_1_1 = '{} is likely to make NCAA Tournament.'.format(college)
        else:
            text3_1_1 = '{} is unlikely to make NCAA Tournament.'.format(college)
                
        text3_1_2 = '{}% Chance'.format(int(mode_NCWA_prob*100))
        text3_1_3 = 'Odds* of {}'.format(odds_NCWA)
        
        ### Tab 3-2: NCWA rank
        data = df_WSRank[df_WSRank.index == college]
        data_count = pd.DataFrame(data.WCWS_Rank.value_counts()/10).reset_index()
        Rank_list = pd.DataFrame()
        Rank_list['Rank'] = ['Champion','Finalist','Semi-Finalist','Top 6','Top 8', 
                             'Top 16', 'Top 32','Top 48','Top 64']
        Rank_list['RankNum'] = [1, 2, 3, 5, 7, 9, 17, 33, 49]
        data_rank = Rank_list.merge(data_count,how='left',left_on='RankNum',right_on='index').fillna(0)
        data_rank['CDF']=np.round(np.cumsum(data_rank.WCWS_Rank),decimals=2)
        data_rank['CDF_perc']=data_rank['CDF'].astype('str')+'%'
        data_rank_all = data_rank.copy()
        data_rank = data_rank.iloc[0:8,:]
        
        fig3_2 = go.Figure(data=[go.Bar(x=data_rank.Rank,y=data_rank.CDF,text=data_rank.CDF)])
        fig3_2.update_traces(texttemplate='%{text:.2s}', textposition='outside', marker_color='#ffa88f')
        fig3_2.update_layout(
            xaxis=dict(showgrid=False, zeroline=False, showticklabels=True),
            yaxis=dict(zeroline=True, gridcolor='white'),
            xaxis_title='NCAA Tournament Rank',
            yaxis_title='Total % Chance',
            #xaxis_range=[0,100],
            yaxis_range=[0,120],
            paper_bgcolor="#f7f7f9",
            plot_bgcolor='#f7f7f9',
            height=300,
            width=670,
            margin=dict(
                l=70,
                r=40,
                b=70,
                t=40,
                pad=4
            ),
            uniformtext_minsize=8, 
            uniformtext_mode='hide',
        )

        mode_NCWA_Rank = data_rank.Rank[data_rank.index == data_rank.WCWS_Rank.idxmax()].item()
        mode_NCWA_Rank_prob = data_rank.WCWS_Rank.max()
        WS_prob = data_rank.CDF[4]
        WS_odds = np.round((WS_prob/100)/(1-WS_prob/100+0.000001),decimals=2)
        if WS_odds > 100:
            WS_odds = 100
            
        text3_2_1 = "{} is most likely to place {}.".format(college,mode_NCWA_Rank)
        text3_2_2 = '{}% Chance'.format(int(mode_NCWA_Rank_prob))
        if WS_odds >= 1:
            text3_2_3 = '{} is likely to make WS (Top 8) Tournament.'.format(college)
        else:
            text3_2_3 = '{} is unlikely to make WS (Top 8) Tournament.'.format(college)
        text3_2_4 = '{}% Chance'.format(int(WS_prob))
        text3_2_5 = 'Odds* of {}'.format(WS_odds)
        
        table3 = [dbc.Table.from_dataframe(data_rank.set_index('Rank')[['CDF_perc']].T, 
                                           striped=True, bordered=True, hover=True,className='text-center')]
        
        ###################
        # Sidebar content #                                                               College Selected - history
        #-----------------#-------------------------------------------------------------------------------------------
        hist_new = dbc.Card([
            dbc.CardBody([
                html.H5("2013-2018 Stats",id = 'College_History',
                        style={'textAlign': 'center'},className="card-title text-center text-success"),
                
                html.Hr(),
                #html.H6("Best Final RPI Ranking",className="text-center text-strong"),
                html.H6("Best Final RPI Ranking : {}".format(int(team_RPI_high[np.where(teams == college)])),
                        id = 'Top Team 2', style={'textAlign': 'left'}),
                html.Br(),
                #html.H6("NCAA/WS Tournament",className="text-center strong"),
                html.H6("WS Championships :  {}".format(int(team_first[np.where(teams == college)])),
                        id = 'Top Team 4', style={'textAlign': 'left'}),
                html.H6("WS Appearances :  {}".format(int(np.nan_to_num(team_WCWS[np.where(teams == college)]))),
                        id = 'Top Team 6',style={'textAlign': 'left'}),
                html.Br(),
                html.H6("Best NCAA Finish :  {}".format(int(np.nan_to_num(team_high[np.where(teams == college)]))),
                        id = 'Top Team 1', style={'textAlign': 'left'}),
                html.H6("Average NCAA Finish :  {}".format(int(np.nan_to_num(team_rank[np.where(teams == college)]))),
                        id = 'Top Team 3', style={'textAlign': 'left'}),
                html.H6("NCAA Appearances :  {}".format(int(np.nan_to_num(team_NCAA[np.where(teams == college)]))),
                        id = 'Top Team 5', style={'textAlign': 'left'}),
            ]),
        ],
        id = 'Card_Update',
        )

        return (text1, fig1, text2, fig2, text2_1, text2_2, text3_1_1, text3_1_2, text3_1_3, fig3_1,
                text3_2_1, text3_2_2, text3_2_3, text3_2_4, text3_2_5, fig3_2, table3, hist_new)            #Return No Selection
    #=================================================================================================================

# Run the app
if __name__ == '__main__':
    app.run_server(mode='external',port=8080,debug=True)
    

Dash app running on http://127.0.0.1:8080/
